# Torchvision object detection finetuning
사전학습된 모델의 미세조정을 해 보자. 예제로서 [Penn-Fudan 데이터셋](https://www.cis.upenn.edu/~jshi/ped_html/)으로 사전학습된 Mask R-CNN을 사용한다. 여기에는 170개 이미지와 345개 보행자 인스턴스가 포함되어 있다. 커스텀 데이터셋으로 인스턴트 분류 모델을 학습시킬 것이다.

## Defining the Dataset
커스텀 데이터셋을 만드는 방법은 다음과 같다.

1. `torch.utils.data.Dataset`을 상속하는 클래스를 생성한다.
2. 데이터 수를 반환하는 함수 `__len__()`를 구현한다.
3. 인덱스에 해당하는 이미지와 타겟을 반환하는 함수 `__getitem__()`을 구현한다. 자세히는,
    - image: 크기 (H, W)인 PIL(Python Imaging Library) 이미지
    - target: 이하의 필드를 갖는 딕셔너리
        - boxes: 바운딩 박스의 모서리 좌표 (x0, y0, x1, y1), FloatTensor[N, 4]
        - labels: 각 박스의 레이블, Int64Ternsor[N], 배경 레이블은 0으로 약속되어 있다.
        - image_id: 이미지 식별자, Int64Tensor[1]
        - area: 바운딩 박스의 면적, Tensor[N]
        - iscrowd: 복수의 객체가 포함되었는지 여부, 평가 시 제외, UInt8Tensor[N]
        - (optionally) masks: 각 객체에 대한 분류 마스크, UInt8Tensor[N, H, W]
        - (optionally) keypoints: 각 객체에 대한 K개 키포인트 (x, y, visibility), FloatTensor[N, K, 3]
4. (optionally) 종횡비가 비슷한 이미지들로 배치를 구성하려면 함수 `get_height_and_width()`를 추가한다.

### Writing a custom dataset for PennFudan
PennFudan 파일을 [다운로드](https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip)하여 사용한다(`./data/PennFudanPed`). 그리고 커스텀 데이터셋을 작성한다.

In [1]:
import os
import numpy as np
import torch
from PIL import Image

class PennFudanDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        
        # note that we haven't converted the mask to RGB, because each color corresponds to a different instance with 0 being background
        mask = Image.open(mask_path)

        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

## Defining your model
본 튜토리얼에서는 사전학습된 Mask R-CNN 모델을 사용한다.

- Instance Segmentation을 위한 딥러닝 프레임워크이다.
- Faster R-CNN 구조에 기반한다.
    1. 이미지로부터 ROI를 예측하고, 다시 각 ROI에 대하여 class와 bounding box를 예측한다.
    2. 여기에 Segmentation mask를 예측하는 분기를 추가한다.

기존 모델을 수정하는 상황은 일반적으로 2가지 경우이다.

- 사전훈련된 모델에서 시작하여 마지막 레이어를 미세조정한다.
- 모델의 백본을 다른 모델의 것으로 교체한다.

여기서는 데이터셋이 작아도 가능한 첫번째 경우로 진행한다.

In [2]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

## Putting everything together
Torchvision 저장소의 [references/detection](https://github.com/pytorch/vision/tree/main/references/detection)에는 감지모델의 학습과 평가를 단순화하도록 돕는 다양한 함수가 제공된다. 이 폴더를 복사해서 사용한다(`./references/detection`).

- 이하의 `get_transform()`은 학습 시 이미지를 확률적으로 반전시킨다. 즉 데이터를 증강한다.

In [3]:
import references.detection.transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

테스트한다.

In [4]:
import references.detection.utils as utils

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = PennFudanDataset('data/PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

# For Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)   # Returns losses and detections

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)            # Returns predictions

메인함수를 작성한다.

In [5]:
from references.detection.engine import train_one_epoch, evaluate
import references.detection.utils as utils

def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 2
    # use our dataset and defined transformations
    dataset = PennFudanDataset('data/PennFudanPed', get_transform(train=True))
    dataset_test = PennFudanDataset('data/PennFudanPed', get_transform(train=False))

    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices[:-50])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")
    
if __name__ == "__main__":
    main()

Epoch: [0]  [ 0/60]  eta: 0:00:28  lr: 0.000090  loss: 4.8592 (4.8592)  loss_classifier: 0.5520 (0.5520)  loss_box_reg: 0.1816 (0.1816)  loss_mask: 4.0992 (4.0992)  loss_objectness: 0.0258 (0.0258)  loss_rpn_box_reg: 0.0007 (0.0007)  time: 0.4731  data: 0.1320  max mem: 1876
Epoch: [0]  [10/60]  eta: 0:00:13  lr: 0.000936  loss: 2.0420 (3.0179)  loss_classifier: 0.3845 (0.3683)  loss_box_reg: 0.2960 (0.2712)  loss_mask: 1.3483 (2.3503)  loss_objectness: 0.0234 (0.0231)  loss_rpn_box_reg: 0.0044 (0.0050)  time: 0.2669  data: 0.0144  max mem: 2852
Epoch: [0]  [20/60]  eta: 0:00:10  lr: 0.001783  loss: 0.9476 (1.9057)  loss_classifier: 0.2306 (0.2629)  loss_box_reg: 0.2665 (0.2551)  loss_mask: 0.4286 (1.3600)  loss_objectness: 0.0194 (0.0219)  loss_rpn_box_reg: 0.0048 (0.0058)  time: 0.2439  data: 0.0026  max mem: 3505
Epoch: [0]  [30/60]  eta: 0:00:07  lr: 0.002629  loss: 0.4945 (1.4555)  loss_classifier: 0.0890 (0.2028)  loss_box_reg: 0.1779 (0.2308)  loss_mask: 0.2268 (0.9992)  loss_ob